In [ ]:
using PyPlot, Statistics, DataFrames, ProgressMeter, Random, CSV

In [ ]:
# Spingenerator
function Spin(L)
    s=zeros(L)
    for i in 1:L
        s[i]=rand([-1,1])
    end
    return s
end

In [ ]:
# Inverser Abstand zu allen anderen Plätzen in einer Matrix gespeichert
function Wechselwirkung(J,L)
    V=zeros(L,L)
    for i in 1:L
        for j in 1:L
            if i!=j
                V[i,j]=J/abs(i-j)^2
            end
        end
    end
    return V
end

In [ ]:
# Ein Schritt des Metropolisalgorithmus
function metropolis_step!(σ,V,β,J)
    # Wahl eines zufälligen Platzes
    x=rand(collect(1:size(σ)[1]))
    # Wechselwirkungsterm mit der Matrix V, die den inversen Abstand enthält 
    W=V[:,x]'*σ
    # Energiedifferenz durch Spinflip
    ΔE=2*σ[x]*W
    # Ziehen einer Zufallszahl r und Akzeptieren der neuen Konfiguration, wenn r<=exp(-β*ΔE)
    r=rand()
    if r<=exp(-β*ΔE)
        σ[x]=-σ[x]
    end
    return σ
end

In [ ]:
# Metropolisalgorithmus mit Speicherung der Konfigurationen
function run_metropolis!(L,β,J,N,init_sweeps,sample_interval)
    Config=zeros(L,N)
    config=Spin(L)
    #Matrix der inversen Abstände definieren
    V=Wechselwirkung(J,L)
    # Thermalisierung
    for i in 1:init_sweeps
        config=metropolis_step!(config,V,β,J)
    end
    # N Konfigurationen mit Zwischenschritten (sample_interval)
    for i in 1:N
        for j in 1:sample_interval
            config=metropolis_step!(config,V,β,J)
        end
        Config[:,i]=config
    end
    return Config
end

In [ ]:
# Dies ist der Test-Metropolisalgorithmus, der die Magnetisierung für generierte Konfigurationen berechnet
function run_metropolis_1!(L,β,J,N,init_sweeps,sample_interval)
    σ = rand([-1,1],L)
    observables = Matrix{Float64}(undef,N,1)
    #Matrix der inversen Abstände definieren
    V=Wechselwirkung(J,L)
    #Thermalisierung
    for i in 1:init_sweeps
        σ=metropolis_step!(σ,V,β,J)
    end
    # N Konfigurationen mit Zwischenschritten (sample_interval)
    for t in 1:N
        for i in 1:sample_interval
            σ=metropolis_step!(σ,V,β,J)
        end
        observables[t,1]=sum(σ)
    end
    return 1/β,mean(observables[:,1]),var(observables[:,1]),mean(abs.(observables[:,1]))
end

In [ ]:
# Daten für die Magnetisierungskurve sammeln
data_for_test= DataFrame(L=Int[], T=Float64[],M_mean=Float64[], M_var=Float64[], M_abs=Float64[])
L = zeros(100)
T_range = range(0.1,3.0, length=100)
M_mean=zeros(100)
M_var=zeros(100)
M_abs=zeros(100)
@showprogress for i in 1:100
    L[i]=100
    β_inv,M_mean[i],M_var[i],M_abs[i]=run_metropolis_1!(500,1/T_range[i],1,6000,1000,10000)
end
for i in 1:100
    push!(data_for_test.L,L[i])
    push!(data_for_test.T,T_range[i])
    push!(data_for_test.M_mean,M_mean[i])
    push!(data_for_test.M_var,M_var[i])
    push!(data_for_test.M_abs,M_abs[i])
end

In [ ]:
# Plot der Magnetiserungskurve in Abhängigkeit von der Temperatur, um einen Phasenübergang zu suchen
plot(data_for_test.T,(data_for_test.M_abs./500).^2,"o")
xticks([0.0,0.5,1.0,1.5,2.0,2.5,3.0])
ax = gca()
ax[:tick_params](labelsize=15)
ax[:set_xlabel]("T",fontsize=15)
ax[:set_ylabel]("(|M|/L)²",fontsize=15)

In [ ]:
# Spinkonfigurationen für die Temperaturen T=2,3,5 generieren
label=Array{String}(undef,10000)
for i in 1:10000
    label[i]=string("Konfiguration ",i)
end
Σ_2=run_metropolis!(500,1/2 ,1.0,10000,1000,10000)
data_2= DataFrame(Σ_2,label)
Σ_3=run_metropolis!(500,1/3 ,1.0,10000,1000,10000)
data_3= DataFrame(Σ_3,label)
Σ_5=run_metropolis!(500,1/5 ,1.0,10000,1000,10000)
data_5= DataFrame(Σ_5,label)

In [ ]:
# Spinkonfigurationen exportieren
CSV.write("...data_2.csv",data_2)
CSV.write("...data_3.csv",data_3)
CSV.write("...data_5.csv",data_5)